# Laplace problem with mixed boundary conditions

We consider the Laplace problem with mixed boundary conditions. On the unit square domain $\Omega = {[0,1]}^2$, the differential problem reads:
\begin{equation*}
  \begin{cases}
    -\nabla\cdot{( k \boldsymbol{\nabla}u)} = f, \qquad & \text{in }\Omega, \\
    u = g_D, \qquad & \text{on }\Gamma_D
      = \{(x,y) \in \mathbb{R}^2 \,|\, x \in \{0, 1\} \text{ and } y \in (0, 1) \}, \\
    -k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}} = g_N, \qquad & \text{on }\Gamma_N
      = \{(x,y) \in \mathbb{R}^2 \,|\, x \in (0, 1) \text{ and } y = 0 \}, \\
    -k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}} = \alpha (u - u_R), \qquad & \text{on }\Gamma_R
      = \{(x,y) \in \mathbb{R}^2 \,|\, x \in (0, 1) \text{ and } y = 1 \},
  \end{cases}
\end{equation*}
with problem data given by
\begin{equation*}
  k = 1, \quad
  \alpha = 1, \quad
  f = 0, \quad
  g_D = 0, \quad
  g_N = -1, \quad
  u_R = 1.
\end{equation*}

1.   Derive the expression for the **bilinear form** and **linear functional** associated to the **weak formulation** of the problem

2.   Solve the problem using the variational approach in FEniCS. Use the space $\mathbb{P}^k$ of continuous piecewise polynomial functions with degree at most $k$.



## Weak formulation

We multiply the equation by a test
function $v$ and we integrate over the domain $\Omega$ to obtain:
\begin{equation*}
  -\int_\Omega \nabla\cdot{( k \boldsymbol{\nabla}u)} v \, 
  d\boldsymbol{x} = \int_\Omega f v \, d\boldsymbol{x},
\end{equation*}
integrating by parts:
\begin{equation*}
  \int_{\partial\Omega} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma +
  \int_\Omega k\boldsymbol{\nabla} u\cdot \boldsymbol{\nabla} v \, d\boldsymbol{x} =
  \int_\Omega f v \, d\vec{x}.
\end{equation*}

The **integral over the boundary $\partial\Omega$ is split into three contributions**:
\begin{equation*}
  \int_{\partial\Omega} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma =
  \int_{\Gamma_D} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma +
  \int_{\Gamma_N} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma +
  \int_{\Gamma_R} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma.
\end{equation*}


*    On the **Dirichlet boundary** $\Gamma_D$ we assume that the test function $v$ is equal to $0$, since the value of $u$ is fixed equal to $0$ (essential BC). 

*    On the **Neumann boundary** $\Gamma_N$ the value of the heat flux is known (natural BC):
\begin{equation*}
  \int_{\Gamma_N} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma =
    \int_{\Gamma_N} g_N v \, d\sigma.
\end{equation*}

*    On the **Robin boundary** $\Gamma_R$ the condition is enforced as in the Neumann case (natural BC)
\begin{equation*}
  \int_{\Gamma_R} (-k \boldsymbol{\nabla} u\cdot{\boldsymbol{n}}) v \, d\sigma =
    \int_{\Gamma_R} \alpha (u - u_R) v \, d\sigma.
\end{equation*}

Then, we reorder the terms to leave the functions depending on $u$ on the left-hand side and the functions depending on the problem data on the right-hand side. Proceeding in this way, we obtain
\begin{equation*}
  \int_\Omega  k\boldsymbol{\nabla} u\cdot \boldsymbol{\nabla} v \, d\boldsymbol{x}
  + \int_{\Gamma_R} \alpha u v \, d\sigma =
  \int_\Omega f v \, d\boldsymbol{x}
  - \int_{\Gamma_N} g_N v \, d\sigma
  + \int_{\Gamma_R} \alpha u_R v \, d\sigma.
\end{equation*}

Therefore the **weak formulation** reads:  
$$
\text{Find } u\in H^1_{0,\Gamma_D}(\Omega) \;\text{ such that } \; a(u, v) = L(v) \quad \forall v\in H^1_{0,\Gamma_D}(\Omega),
$$ 
where the **bilinear form** is defined by
\begin{equation*}
  a(u, v) = \int_\Omega k\boldsymbol{\nabla} u\cdot \boldsymbol{\nabla} v \, d\boldsymbol{x}
          + \int_{\Gamma_R} \alpha u v \, d\sigma,
\end{equation*}
and the **linear functional** is defined such that
\begin{equation*}
  L(v) = \int_\Omega f v \, d\boldsymbol{x} 
       - \int_{\Gamma_N} g_N v \, d\sigma
       + \int_{\Gamma_R} \alpha u_R v \, d\sigma.
\end{equation*}

#### **Exercise**: Verify that the assumptions of the Lax-Milgram theorem are satisfied (assuming sufficient regularity of the data). Is the bilinear function $a$ symmetric?

## Numerical solution with FEniCS

In [ ]:
%%capture
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *

First, we generate the mesh.

In [ ]:
# 1a. generate the mesh
nx, ny = 80, 80
mesh = UnitSquareMesh(nx, ny, 'crossed')
dim = mesh.geometric_dimension()
print(dim)

2


In this case, since we have to deal with different types of boundary condition, we have to **distinguish the different portion of the boundary $\partial\Omega$**

In [ ]:
# 1b. mark the boundaries
boundary_markers = MeshFunction('size_t', mesh, dim-1, 0)
boundary_markers.rename('boundary label', 'boundary')

# Neumann boundary
class BottomBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[1], 0.0)

# Robin boundary
class TopBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[1], 1.0)

# Dirichlet boundaries
class LeftBoundary(SubDomain):
    def inside (self, x, on_boudary):
        return on_boudary and near(x[0], 0.0)

class RightBoundary(SubDomain):
    def inside (self, x, on_boudary):
        return on_boudary and near(x[0], 1.0)


bottom = BottomBoundary()
top = TopBoundary()
left = LeftBoundary()
right = RightBoundary()

# we assign an integer label for each portion of the boundary
bottom.mark(boundary_markers, 1)
right.mark(boundary_markers, 2)
top.mark(boundary_markers, 3)
left.mark(boundary_markers, 4)

In [ ]:
# to understand built-in classes and function we can look on the FEniCS manual or use the help
help(MeshFunction)

help(near)
near(0.5, 0.45, 0.1)
#near(0.5, 0.4, 0.05)

Help on class MeshFunction in module dolfin.mesh.meshfunction:

class MeshFunction(builtins.object)
 |  Static methods defined here:
 |  
 |  __new__(cls, value_type, mesh, dim, value=None)
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)

Help on built-in function near in module dolfin.cpp.math:

near(...) method of builtins.PyCapsule instance
    near(x0: float, x1: float, eps: float=3e-16) -> bool



True

Since we need to perform integrals over the boundary edges we need to define the Lesbegue measure *ds*:

In [ ]:
# 1c. setting up the measures
dx = Measure('dx', domain=mesh)
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

Now we can move to the second step of the FEM method implemented with FEniCS, namely the definition of the discrete space: 

In [ ]:
# 2a. definition of finite element space
degree = 1
V = FunctionSpace(mesh, 'CG', degree)

As usual **homogeneous Dirichlet conditions** can be enforced in a **strong way** by fixing some of the DOFs of the discrete space to zero. 

In [ ]:
# 2b. definition of Dirichlet boundary conditions
gD = Constant(0.0)

bc = [
    DirichletBC(V, gD, boundary_markers, 2),
    DirichletBC(V, gD, boundary_markers, 4)]

Now we can move to the main part of the program. i.e. the **definition and assembling of the discrete problem**. The FEniCS formulation corresponds exactly to the weak form defined above.

In [ ]:
# 3a. definition of bilinear forms, linear functionals 
k = Constant(1.0)
alpha = Constant(1.0)

f = Constant(0.0)
gN = Constant(-1.0)
uR = Constant(1.0)

u = TrialFunction(V)
v = TestFunction(V)

a = k * dot(grad(u), grad(v)) * dx + alpha * u * v * ds(3)
L = f * v * dx - gN * v * ds(1) + alpha * uR * v * ds(3)

# 3b. assembling the discrete problem
A, b = assemble_system(a, L, bc)

Calling FFC just-in-time (JIT) compiler, this may take some time.


### Iterative algebraic solvers 

Notice above that we did not solve the discrete system yet, but we just perform the assembling. Since the problem considered here is a bit more involved than the Laplace problem of Ex 1, we use an **iterative solver** instead of the built-in direct solver provided by *solve(a == L, u, bc)*.

In [ ]:
# 4b. solve the system
u = Function(V)
u.rename('temperature', 'temperature')

x = u.vector()

# Timer() is used to measure the computational time
timer = Timer()

# There are different choices for the parameters to set up the conjugate gradient solver
precon = 'none'
#precon = 'amg'
solver = KrylovSolver('cg', precon)
solver.parameters["relative_tolerance"] = 5e-8
#solver.parameters["maximum_iterations"] = 1000
#solver.parameters['monitor_convergence'] = True

solver.solve(A, x, b)

elapsed, _, _ = timer.elapsed()
print('Time to solve: {:.3f}s'.format(elapsed))

Time to solve: 0.080s


Finally, we conclude with the post-processing. In this case we export the solution for visualisation in Paraview.

In [ ]:
# 5a. save solution
vtkfile = File('ex02.pvd')
vtkfile << u

#### **Exercise**: Perform the convergence analysis as we have done for the homogeneous Dirichlet problem. Take again $k=1$, $\alpha=1$, and as exact solution $u(x, y) = \sin{(\pi x)} \sin{(\pi y)}$. Infer the load function $f$ and the boundary data ($g_D, g_N, u_R$) from the exact solution $u$.

In [ ]:
def solver(n, degree):
  nx, ny = n, n
  mesh = UnitSquareMesh(nx, ny, 'crossed')
  dim = mesh.geometric_dimension()

  boundary_markers = MeshFunction('size_t', mesh, dim-1, 0)
  boundary_markers.rename('boundary label', 'boundary')

  class BottomBoundary(SubDomain): # Neumann boundary
    def inside(self, x, on_boundary):
      return on_boundary and near(x[1], 0)

  class TopBoundary(SubDomain): # Robin boundary
    def inside(self, x, on_boundary):
      return on_boundary and near(x[1], 1)

  class LeftBoundary(SubDomain): # Dirichlet boundary
    def inside(self, x, on_boundary):
      return on_boundary and near(x[0], 0)

  class RightBoundary(SubDomain): # Dirichlet boundary
    def inside(self, x, on_boundary):
      return on_boundary and near(x[0], 1)

  bottom = BottomBoundary()
  top = TopBoundary()
  left = LeftBoundary()
  right = RightBoundary()

  bottom.mark(boundary_markers, 1)
  right.mark(boundary_markers, 2)
  top.mark(boundary_markers, 3)
  left.mark(boundary_markers, 4)

  V = FunctionSpace(mesh, 'CG', degree)

  gD = Constant(0.0)
  bc = [DirichletBC(V, gD, boundary_markers, 2),
        DirichletBC(V, gD, boundary_markers, 4)]

  #dx = Measure('dx', domain=mesh)
  ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

  u = TrialFunction(V)
  v = TestFunction(V)
  
  k = Constant(1.0)
  alpha = Constant(1.0)

  f = Expression('2*pi*pi*sin(pi*x[0])*sin(pi*x[1])', degree=degree+1)
  gN = Expression('pi*sin(x[0])', degree=degree+1)
  uR = Expression('-pi*sin(x[0])', degree=degree+1)

  a = k*inner(grad(u), grad(v))*dx + alpha*u*v*ds(3)
  L = f*v*dx - gN*v*ds(1) + alpha*uR*v*ds(3)

  A, b = assemble_system(a, L, bc)

  u = Function(V)
  x = u.vector()

  precon = 'none'
  tol = 5e-8
  solver = KrylovSolver('cg', precon)
  solver.parameters["relative_tolerance"] = tol
  solver.solve(A, x, b)

  return u

In [ ]:
import numpy as np

M = np.zeros((4,5))

for degree in [1, 2]:
  u = Expression('sin(pi*x[0])*sin(pi*x[1])', degree=degree+2)
  f = Expression('2*pi*pi*sin(pi*x[0])*sin(pi*x[1])', degree=degree+1)

  gN = Expression('pi*sin(x[0])', degree=degree+1)
  uR = Expression('-pi*sin(x[0])', degree=degree+1)

  for j in range(5):
    n = 10*(2**j)

    uh = solver(n, degree)

    errL2 = errornorm(u, uh, 'L2')
    errH1 = errornorm(u, uh, 'H10')

    i = 2*(degree-1)
    M[i, j] = errL2
    M[i+1, j] = errH1

    print('n={} degree={} eL2={:.2e} eH1={:.2e}'.format(n, degree, errL2, errH1));

print()

n=10 degree=1 eL2=1.68e-01 eH1=7.77e-01
n=20 degree=1 eL2=1.70e-01 eH1=7.73e-01
n=40 degree=1 eL2=1.70e-01 eH1=7.73e-01
n=80 degree=1 eL2=1.70e-01 eH1=7.74e-01
n=160 degree=1 eL2=1.70e-01 eH1=7.74e-01
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
n=10 degree=2 eL2=1.70e-01 eH1=7.72e-01
n=20 degree=2 eL2=1.70e-01 eH1=7.74e-01
n=40 degree=2 eL2=1.70e-01 eH1=7.74e-01
n=80 degree=2 eL2=1.70e-01 eH1=7.74e-01
n=160 degree=2 eL2=1.70e-01 eH1=7.74e-01



In [ ]:
import matplotlib.pyplot as plt

nx, ny = 10, 10
mesh = UnitSquareMesh(nx, ny, 'crossed')
dim = mesh.geometric_dimension()

boundary_markers = MeshFunction('size_t', mesh, dim-1, 0)
boundary_markers.rename('boundary label', 'boundary')

class BottomBoundary(SubDomain): # Neumann boundary
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1], 0)

class TopBoundary(SubDomain): # Robin boundary
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1], 1)

class LeftBoundary(SubDomain): # Dirichlet boundary
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0], 0)

class RightBoundary(SubDomain): # Dirichlet boundary
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0], 1)

bottom = BottomBoundary()
top = TopBoundary()
left = LeftBoundary()
right = RightBoundary()

bottom.mark(boundary_markers, 1)
right.mark(boundary_markers, 2)
top.mark(boundary_markers, 3)
left.mark(boundary_markers, 4)

degree = 1
V = FunctionSpace(mesh, 'CG', degree)

gD = Constant(0.0)
bc = [DirichletBC(V, gD, boundary_markers, 2),
      DirichletBC(V, gD, boundary_markers, 4)]

dx = Measure('dx', domain=mesh)
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

u = TrialFunction(V)
v = TestFunction(V)
  
k = Constant(1.0)
alpha = Constant(1.0)

f = Expression('2*pi*pi*sin(pi*x[0])*sin(pi*x[1])', degree=degree+1)
gN = Expression('pi*sin(x[0])', degree=degree+1)
uR = Expression('-pi*sin(x[0])', degree=degree+1)

a = k*dot(grad(u), grad(v))*dx + alpha*u*v*ds(3)
L = f*v*dx - gN*v*ds(1) + alpha*uR*v*ds(3)

A, b = assemble_system(a, L, bc)

u = Function(V)
x = u.vector()

precon = 'none'
tol = 5e-8
solver = KrylovSolver('cg', precon)
solver.parameters["relative_tolerance"] = tol
solver.solve(A, x, b)

u = Expression('sin(pi*x[0])*sin(pi*x[1])', degree=degree+2)

#plot(x)
#plot(mesh)
#plt.show()

Calling FFC just-in-time (JIT) compiler, this may take some time.
